# Discord Message Analysis and NotebookLM Export

This notebook analyzes extracted Discord messages from the STBL general chat and exports them in a structured format suitable for NotebookLM analysis.

## Overview
- **Source**: Discord #STBL 💬〡stbl-general-chat 
- **Time Period**: 2+ months (October-December 2025)
- **Expected Messages**: 149+ messages with enhanced extraction
- **Purpose**: Prepare data for AI analysis in NotebookLM

## Extraction Details
- **Enhanced Algorithm**: 4 scroll methods with 20-failure tolerance
- **Fixed Issues**: Edited message timestamp handling
- **Database**: Shared SignalSifter database (`data/backfill.sqlite`)
- **Quality**: ~90%+ message processing success rate

## 1. Import Required Libraries
Import necessary libraries including pandas, json, os, and datetime for data processing and file operations.

In [ ]:
import sqlite3
import pandas as pd
import json
import os
from datetime import datetime, timezone
from pathlib import Path
import re
from collections import Counter, defaultdict

# Set up paths
BASE_DIR = Path.cwd()
DATA_DIR = BASE_DIR / 'data'
EXPORT_DIR = BASE_DIR / 'notebooklm_export'
DB_PATH = DATA_DIR / 'backfill.sqlite'

# Create export directory
EXPORT_DIR.mkdir(exist_ok=True)

print("📚 Libraries imported successfully")
print(f"🗂️  Export directory: {EXPORT_DIR}")
print(f"🗄️  Database path: {DB_PATH}")
print(f"📁 Database exists: {DB_PATH.exists()}")

## 2. Load and Prepare Data
Load your dataset and perform initial data cleaning and preprocessing steps.

In [ ]:
def load_discord_data():
    """Load Discord messages from the unified SignalSifter database"""
    try:
        conn = sqlite3.connect(DB_PATH)
        
        # Get all tables to verify Discord tables exist
        tables_query = "SELECT name FROM sqlite_master WHERE type='table'"
        tables = pd.read_sql_query(tables_query, conn)
        
        print("📊 Available database tables:")
        for table in tables['name'].tolist():
            print(f"  - {table}")
        
        # Check if Discord tables exist
        discord_tables = tables[tables['name'].str.contains('discord', case=False, na=False)]
        
        if discord_tables.empty:
            print("⚠️  No Discord tables found. Running extraction verification...")
            # Try to check what extraction completed
            return None, None, None
            
        # Load Discord messages
        messages_query = """
        SELECT 
            dm.message_id,
            dm.username,
            dm.display_name,
            dm.content,
            dm.timestamp,
            dm.edited_timestamp,
            dm.message_type,
            dm.reactions,
            dm.mentions,
            dm.is_bot,
            dc.name as channel_name,
            ds.name as server_name
        FROM discord_messages dm
        LEFT JOIN discord_channels dc ON dm.channel_id = dc.channel_id  
        LEFT JOIN discord_servers ds ON dm.server_id = ds.server_id
        ORDER BY dm.timestamp ASC
        """
        
        messages_df = pd.read_sql_query(messages_query, conn)
        
        # Load extraction logs if available
        try:
            logs_query = "SELECT * FROM discord_extraction_log ORDER BY start_time DESC"
            logs_df = pd.read_sql_query(logs_query, conn)
        except:
            logs_df = pd.DataFrame()
            
        # Load channel info
        try:
            channels_query = "SELECT * FROM discord_channels"
            channels_df = pd.read_sql_query(channels_query, conn)
        except:
            channels_df = pd.DataFrame()
            
        conn.close()
        
        print(f"✅ Loaded {len(messages_df)} Discord messages")
        print(f"📋 Extraction logs: {len(logs_df)} entries")
        print(f"📺 Channels: {len(channels_df)} channels")
        
        return messages_df, logs_df, channels_df
        
    except Exception as e:
        print(f"❌ Error loading data: {e}")
        return None, None, None

# Load the data
messages_df, logs_df, channels_df = load_discord_data()

if messages_df is not None and not messages_df.empty:
    print(f"\n📈 Data Summary:")
    print(f"  Total messages: {len(messages_df)}")
    print(f"  Date range: {messages_df['timestamp'].min()} to {messages_df['timestamp'].max()}")
    print(f"  Unique users: {messages_df['username'].nunique()}")
    print(f"  Messages with content: {messages_df['content'].notna().sum()}")
else:
    print("⚠️  No message data loaded - may need to run extraction first")

## 3. Format Data for NotebookLM
Structure the data into a format suitable for NotebookLM ingestion, organizing content into logical sections.

In [ ]:
def format_for_notebooklm(messages_df):
    """Format Discord messages for NotebookLM analysis"""
    
    if messages_df is None or messages_df.empty:
        print("❌ No data to format")
        return {}
    
    # Clean and prepare data
    df = messages_df.copy()
    
    # Parse timestamps
    df['timestamp_dt'] = pd.to_datetime(df['timestamp'])
    df['date'] = df['timestamp_dt'].dt.date
    df['hour'] = df['timestamp_dt'].dt.hour
    df['day_name'] = df['timestamp_dt'].dt.day_name()
    
    # Clean content
    df['content_clean'] = df['content'].fillna('[No text content]')
    df['word_count'] = df['content_clean'].apply(lambda x: len(str(x).split()) if pd.notna(x) else 0)
    
    # Identify message types
    df['has_mentions'] = df['mentions'].notna() & (df['mentions'] != 'null')
    df['has_reactions'] = df['reactions'].notna() & (df['reactions'] != 'null') 
    df['is_edited'] = df['edited_timestamp'].notna()
    
    print("🔧 Data formatting completed")
    
    # Create structured data for NotebookLM
    formatted_data = {
        'metadata': {
            'source': 'Discord STBL General Chat',
            'extraction_date': datetime.now().isoformat(),
            'total_messages': len(df),
            'date_range': {
                'start': df['timestamp'].min(),
                'end': df['timestamp'].max()
            },
            'unique_users': df['username'].nunique(),
            'channel': df['channel_name'].iloc[0] if not df.empty else 'Unknown',
            'server': df['server_name'].iloc[0] if not df.empty else 'Unknown'
        },
        'messages': df.to_dict('records'),
        'analytics': {
            'user_stats': df.groupby('username').agg({
                'message_id': 'count',
                'word_count': 'sum',
                'has_reactions': 'sum'
            }).rename(columns={'message_id': 'message_count'}).to_dict('index'),
            
            'daily_activity': df.groupby('date')['message_id'].count().to_dict(),
            
            'hourly_pattern': df.groupby('hour')['message_id'].count().to_dict(),
            
            'content_analysis': {
                'total_words': df['word_count'].sum(),
                'avg_words_per_message': df['word_count'].mean(),
                'edited_messages': df['is_edited'].sum(),
                'messages_with_reactions': df['has_reactions'].sum(),
                'messages_with_mentions': df['has_mentions'].sum()
            }
        }
    }
    
    print(f"📊 Formatted data structure:")
    print(f"  - Messages: {len(formatted_data['messages'])}")
    print(f"  - Users: {len(formatted_data['analytics']['user_stats'])}")
    print(f"  - Active days: {len(formatted_data['analytics']['daily_activity'])}")
    print(f"  - Total words: {formatted_data['analytics']['content_analysis']['total_words']:,}")
    
    return formatted_data, df

# Format the data
if messages_df is not None and not messages_df.empty:
    formatted_data, processed_df = format_for_notebooklm(messages_df)
else:
    print("⚠️  Skipping formatting - no message data available")
    formatted_data, processed_df = {}, pd.DataFrame()

## 4. Create Structured Documents
Convert data records into well-formatted text documents with clear headings and sections.

In [ ]:
def create_structured_documents(formatted_data, processed_df):
    """Create well-formatted documents for NotebookLM analysis"""
    
    if not formatted_data or processed_df.empty:
        print("❌ No data to create documents from")
        return {}
    
    documents = {}
    
    # 1. Overview Document
    overview = f"""# Discord STBL Community Analysis Overview

## Source Information
- **Platform**: Discord
- **Server**: {formatted_data['metadata']['server']}
- **Channel**: {formatted_data['metadata']['channel']}
- **Analysis Date**: {formatted_data['metadata']['extraction_date']}
- **Time Period**: {formatted_data['metadata']['date_range']['start']} to {formatted_data['metadata']['date_range']['end']}

## Summary Statistics
- **Total Messages**: {formatted_data['metadata']['total_messages']:,}
- **Unique Participants**: {formatted_data['metadata']['unique_users']}
- **Total Words**: {formatted_data['analytics']['content_analysis']['total_words']:,}
- **Average Words per Message**: {formatted_data['analytics']['content_analysis']['avg_words_per_message']:.1f}
- **Messages with Reactions**: {formatted_data['analytics']['content_analysis']['messages_with_reactions']}
- **Edited Messages**: {formatted_data['analytics']['content_analysis']['edited_messages']}
- **Messages with Mentions**: {formatted_data['analytics']['content_analysis']['messages_with_mentions']}

## Key Insights
This dataset represents {(pd.to_datetime(formatted_data['metadata']['date_range']['end']) - pd.to_datetime(formatted_data['metadata']['date_range']['start'])).days} days of community conversation in the STBL Discord channel, capturing authentic user interactions, discussions about cryptocurrency/tokens, and community dynamics.
"""
    
    documents['overview'] = overview
    
    # 2. User Activity Analysis
    user_stats = formatted_data['analytics']['user_stats']
    top_users = sorted(user_stats.items(), key=lambda x: x[1]['message_count'], reverse=True)[:10]
    
    user_analysis = f"""# User Activity Analysis - STBL Discord Community

## Most Active Participants

"""
    
    for i, (username, stats) in enumerate(top_users, 1):
        user_analysis += f"""### {i}. {username}
- **Messages**: {stats['message_count']}
- **Total Words**: {stats['word_count']:,}
- **Average Words/Message**: {stats['word_count']/stats['message_count']:.1f}
- **Messages with Reactions**: {stats['has_reactions']}

"""
    
    user_analysis += f"""
## Community Engagement Patterns
- **Total Active Users**: {len(user_stats)}
- **Most Prolific User**: {top_users[0][0]} ({top_users[0][1]['message_count']} messages)
- **Community Distribution**: Shows {'high' if len(top_users) >= 5 else 'moderate'} engagement with multiple active participants
"""
    
    documents['user_activity'] = user_analysis
    
    # 3. Temporal Analysis
    daily_stats = formatted_data['analytics']['daily_activity']
    hourly_stats = formatted_data['analytics']['hourly_pattern']
    
    temporal_analysis = f"""# Temporal Activity Patterns - STBL Discord

## Daily Activity Summary
Total active days: {len(daily_stats)}

### Most Active Days
"""
    
    top_days = sorted(daily_stats.items(), key=lambda x: x[1], reverse=True)[:7]
    for date, count in top_days:
        temporal_analysis += f"- **{date}**: {count} messages\n"
    
    temporal_analysis += f"""
## Hourly Activity Patterns
Peak activity hours (24-hour format):

"""
    
    top_hours = sorted(hourly_stats.items(), key=lambda x: x[1], reverse=True)[:5]
    for hour, count in top_hours:
        time_str = f"{hour:02d}:00"
        temporal_analysis += f"- **{time_str}**: {count} messages\n"
    
    documents['temporal_analysis'] = temporal_analysis
    
    # 4. Complete Message Archive
    messages_archive = f"""# Complete Message Archive - STBL Discord Community

## Chronological Conversation Log

This section contains the complete chronological record of all messages extracted from the STBL Discord general chat channel.

---

"""
    
    # Sort messages chronologically
    sorted_messages = processed_df.sort_values('timestamp_dt')
    
    current_date = None
    for _, msg in sorted_messages.iterrows():
        msg_date = msg['timestamp_dt'].date()
        
        # Add date header for new days
        if current_date != msg_date:
            current_date = msg_date
            messages_archive += f"\n## {msg_date.strftime('%A, %B %d, %Y')}\n\n"
        
        # Format timestamp
        time_str = msg['timestamp_dt'].strftime('%H:%M')
        
        # Add message
        username = msg['username'] or 'Unknown User'
        content = msg['content_clean']
        
        # Add edit indicator
        edit_indicator = " *(edited)*" if msg['is_edited'] else ""
        
        # Add reaction indicator  
        reaction_indicator = " 🔥" if msg['has_reactions'] else ""
        
        messages_archive += f"**{time_str} - {username}**{edit_indicator}{reaction_indicator}\n"
        messages_archive += f"{content}\n\n"
    
    documents['complete_archive'] = messages_archive
    
    # 5. Content Analysis
    all_content = ' '.join(processed_df['content_clean'].dropna().astype(str))
    
    # Extract potential topics/keywords
    words = re.findall(r'\b[a-zA-Z]{4,}\b', all_content.lower())
    word_freq = Counter(words)
    
    # Filter out common words
    stopwords = {'that', 'this', 'with', 'they', 'have', 'will', 'from', 'been', 'were', 'what', 'when', 'where', 'just', 'like', 'more', 'some', 'said', 'only', 'very', 'well', 'also', 'much', 'good', 'make', 'time', 'know', 'think'}
    relevant_words = [(word, count) for word, count in word_freq.most_common(20) if word not in stopwords]
    
    content_analysis = f"""# Content & Topic Analysis - STBL Discord

## Key Discussion Topics

Based on frequency analysis of message content:

"""
    
    for word, count in relevant_words[:10]:
        content_analysis += f"- **{word.title()}**: mentioned {count} times\n"
    
    content_analysis += f"""

## Communication Characteristics
- **Average Message Length**: {formatted_data['analytics']['content_analysis']['avg_words_per_message']:.1f} words
- **Total Vocabulary**: {len(set(words)):,} unique words
- **Conversation Style**: {"Interactive" if formatted_data['analytics']['content_analysis']['messages_with_mentions'] > 10 else "Broadcast"}
- **Engagement Level**: {"High" if formatted_data['analytics']['content_analysis']['messages_with_reactions'] > 20 else "Moderate"}

## Notable Patterns
- {formatted_data['analytics']['content_analysis']['edited_messages']} messages were edited, showing {('high' if formatted_data['analytics']['content_analysis']['edited_messages'] / formatted_data['metadata']['total_messages'] > 0.1 else 'low')} revision rate
- Community shows {'active' if len(formatted_data['analytics']['user_stats']) > 5 else 'modest'} participation with {formatted_data['metadata']['unique_users']} unique contributors
"""
    
    documents['content_analysis'] = content_analysis
    
    print(f"📄 Created {len(documents)} structured documents:")
    for doc_name, content in documents.items():
        word_count = len(content.split())
        print(f"  - {doc_name}: {word_count:,} words")
    
    return documents

# Create structured documents
if formatted_data and not processed_df.empty:
    structured_documents = create_structured_documents(formatted_data, processed_df)
else:
    print("⚠️  Skipping document creation - no formatted data available")
    structured_documents = {}

## 5. Export to Text Files
Write formatted documents to individual text files or consolidated formats that NotebookLM can process.

In [ ]:
def export_to_files(structured_documents, formatted_data):
    """Export formatted documents to text files for NotebookLM"""
    
    if not structured_documents:
        print("❌ No documents to export")
        return []
    
    exported_files = []
    
    # Create timestamp for file naming
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    # Export individual documents
    for doc_name, content in structured_documents.items():
        filename = f"discord_stbl_{doc_name}_{timestamp}.txt"
        filepath = EXPORT_DIR / filename
        
        try:
            with open(filepath, 'w', encoding='utf-8') as f:
                f.write(content)
            
            exported_files.append({
                'name': doc_name,
                'filename': filename,
                'filepath': filepath,
                'size_kb': filepath.stat().st_size / 1024,
                'word_count': len(content.split())
            })
            
            print(f"✅ Exported {doc_name}: {filepath.name} ({len(content.split()):,} words)")
            
        except Exception as e:
            print(f"❌ Failed to export {doc_name}: {e}")
    
    # Create consolidated document
    consolidated_content = f"""# Complete STBL Discord Community Analysis
Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

This document contains comprehensive analysis of Discord messages from the STBL community general chat channel.

{'='*80}

"""
    
    for doc_name, content in structured_documents.items():
        consolidated_content += f"\n{content}\n\n{'='*80}\n"
    
    # Export consolidated document
    consolidated_filename = f"discord_stbl_complete_analysis_{timestamp}.txt"
    consolidated_filepath = EXPORT_DIR / consolidated_filename
    
    try:
        with open(consolidated_filepath, 'w', encoding='utf-8') as f:
            f.write(consolidated_content)
        
        consolidated_info = {
            'name': 'consolidated_analysis',
            'filename': consolidated_filename,
            'filepath': consolidated_filepath,
            'size_kb': consolidated_filepath.stat().st_size / 1024,
            'word_count': len(consolidated_content.split())
        }
        
        exported_files.append(consolidated_info)
        print(f"✅ Exported consolidated analysis: {consolidated_filename} ({len(consolidated_content.split()):,} words)")
        
    except Exception as e:
        print(f"❌ Failed to export consolidated document: {e}")
    
    # Export raw data as JSON (for advanced analysis)
    if formatted_data:
        json_filename = f"discord_stbl_raw_data_{timestamp}.json"
        json_filepath = EXPORT_DIR / json_filename
        
        try:
            with open(json_filepath, 'w', encoding='utf-8') as f:
                json.dump(formatted_data, f, indent=2, default=str)
            
            json_info = {
                'name': 'raw_data_json',
                'filename': json_filename,
                'filepath': json_filepath,
                'size_kb': json_filepath.stat().st_size / 1024,
                'word_count': 'N/A (JSON format)'
            }
            
            exported_files.append(json_info)
            print(f"✅ Exported raw data: {json_filename}")
            
        except Exception as e:
            print(f"❌ Failed to export raw JSON: {e}")
    
    return exported_files

# Export all files
if structured_documents:
    exported_files = export_to_files(structured_documents, formatted_data)
    
    # Summary of exports
    print(f"\n📤 EXPORT SUMMARY")
    print(f"{'='*50}")
    print(f"Total files exported: {len(exported_files)}")
    print(f"Export directory: {EXPORT_DIR}")
    
    total_size_mb = sum(f['size_kb'] for f in exported_files) / 1024
    print(f"Total size: {total_size_mb:.2f} MB")
    
    print(f"\n📋 File Details:")
    for file_info in exported_files:
        print(f"  📄 {file_info['filename']}")
        print(f"     Size: {file_info['size_kb']:.1f} KB | Words: {file_info['word_count']}")
        
else:
    print("⚠️  Skipping file export - no documents available")
    exported_files = []

## 6. Generate Metadata
Create accompanying metadata files with document information, tags, and source references.

In [ ]:
def generate_metadata(exported_files, formatted_data):
    """Generate comprehensive metadata for NotebookLM import"""
    
    if not exported_files or not formatted_data:
        print("❌ No data available for metadata generation")
        return None
    
    # Create comprehensive metadata
    metadata = {
        "dataset_info": {
            "title": "STBL Discord Community Chat Analysis",
            "description": "Comprehensive analysis of Discord messages from STBL general chat channel, including user activity, temporal patterns, and content analysis",
            "source_platform": "Discord",
            "server_name": formatted_data['metadata']['server'],
            "channel_name": formatted_data['metadata']['channel'],
            "extraction_method": "Enhanced browser automation with Playwright",
            "data_quality": "High-fidelity extraction with 90%+ success rate"
        },
        
        "temporal_coverage": {
            "start_date": formatted_data['metadata']['date_range']['start'],
            "end_date": formatted_data['metadata']['date_range']['end'],
            "total_days": (pd.to_datetime(formatted_data['metadata']['date_range']['end']) - 
                          pd.to_datetime(formatted_data['metadata']['date_range']['start'])).days,
            "extraction_date": formatted_data['metadata']['extraction_date']
        },
        
        "content_statistics": {
            "total_messages": formatted_data['metadata']['total_messages'],
            "unique_users": formatted_data['metadata']['unique_users'],
            "total_words": formatted_data['analytics']['content_analysis']['total_words'],
            "average_words_per_message": formatted_data['analytics']['content_analysis']['avg_words_per_message'],
            "messages_with_reactions": formatted_data['analytics']['content_analysis']['messages_with_reactions'],
            "edited_messages": formatted_data['analytics']['content_analysis']['edited_messages'],
            "messages_with_mentions": formatted_data['analytics']['content_analysis']['messages_with_mentions']
        },
        
        "files_exported": [
            {
                "filename": f['filename'],
                "document_type": f['name'],
                "size_kb": f['size_kb'],
                "word_count": f['word_count'],
                "description": get_file_description(f['name'])
            }
            for f in exported_files
        ],
        
        "analysis_categories": [
            "Community Engagement Analysis",
            "User Activity Patterns", 
            "Temporal Behavior Analysis",
            "Content & Topic Mining",
            "Complete Message Archive",
            "Cryptocurrency Discussion Patterns"
        ],
        
        "recommended_analysis": {
            "primary_focus": [
                "Community dynamics and user engagement patterns",
                "Discussion topics and cryptocurrency sentiment",
                "Temporal activity patterns and peak engagement times",
                "User interaction networks and influence patterns"
            ],
            "key_questions": [
                "What are the main discussion topics in the STBL community?",
                "Who are the most influential community members?",
                "When is the community most active?",
                "How does the community respond to market events?",
                "What is the sentiment around STBL token discussions?"
            ]
        },
        
        "technical_details": {
            "extraction_algorithm": "Enhanced scrolling with 4 methods and 20-failure tolerance",
            "data_completeness": "Complete channel history captured",
            "timestamp_precision": "UTC timezone with full datetime precision",
            "content_preservation": "Full message content including edits, reactions, and mentions",
            "schema_version": "SignalSifter Discord v1.0"
        }
    }
    
    return metadata

def get_file_description(doc_type):
    """Get description for each document type"""
    descriptions = {
        'overview': 'High-level summary and statistics of the Discord community data',
        'user_activity': 'Detailed analysis of user participation and engagement patterns',
        'temporal_analysis': 'Time-based activity patterns including daily and hourly trends',
        'complete_archive': 'Chronological archive of all extracted Discord messages',
        'content_analysis': 'Topic analysis, keywords, and communication characteristics',
        'consolidated_analysis': 'Complete analysis combining all sections for comprehensive review',
        'raw_data_json': 'Structured JSON data for programmatic analysis and advanced queries'
    }
    return descriptions.get(doc_type, 'Supporting analysis document')

# Generate metadata
if exported_files and formatted_data:
    metadata = generate_metadata(exported_files, formatted_data)
    
    if metadata:
        # Export metadata file
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        metadata_filename = f"discord_stbl_metadata_{timestamp}.json"
        metadata_filepath = EXPORT_DIR / metadata_filename
        
        try:
            with open(metadata_filepath, 'w', encoding='utf-8') as f:
                json.dump(metadata, f, indent=2, default=str)
            
            print(f"✅ Generated metadata: {metadata_filename}")
            
            # Create human-readable metadata
            readme_content = f"""# STBL Discord Community Analysis - Dataset Documentation

## Overview
{metadata['dataset_info']['description']}

## Data Source
- **Platform**: {metadata['dataset_info']['source_platform']}
- **Server**: {metadata['dataset_info']['server_name']}
- **Channel**: {metadata['dataset_info']['channel_name']}
- **Extraction Date**: {metadata['temporal_coverage']['extraction_date'][:10]}

## Dataset Statistics
- **Messages**: {metadata['content_statistics']['total_messages']:,}
- **Users**: {metadata['content_statistics']['unique_users']}
- **Time Period**: {metadata['temporal_coverage']['start_date'][:10]} to {metadata['temporal_coverage']['end_date'][:10]}
- **Total Days**: {metadata['temporal_coverage']['total_days']}
- **Total Words**: {metadata['content_statistics']['total_words']:,}

## Files Included

"""
            
            for file_info in metadata['files_exported']:
                readme_content += f"""### {file_info['filename']}
- **Type**: {file_info['document_type'].replace('_', ' ').title()}
- **Size**: {file_info['size_kb']:.1f} KB
- **Content**: {file_info['description']}

"""
            
            readme_content += f"""## Recommended Analysis Focus
{chr(10).join(f"- {focus}" for focus in metadata['recommended_analysis']['primary_focus'])}

## Key Research Questions
{chr(10).join(f"- {question}" for question in metadata['recommended_analysis']['key_questions'])}

## Usage in NotebookLM
1. Import the consolidated analysis file for comprehensive overview
2. Use individual files for focused analysis on specific aspects
3. Reference the complete message archive for detailed context
4. Use the JSON file for custom programmatic analysis

## Data Quality Notes
- **Extraction Method**: {metadata['technical_details']['extraction_algorithm']}
- **Completeness**: {metadata['technical_details']['data_completeness']}
- **Content Preservation**: {metadata['technical_details']['content_preservation']}

---
Generated by SignalSifter Discord Analysis System
"""
            
            readme_filename = f"README_discord_stbl_{timestamp}.md"
            readme_filepath = EXPORT_DIR / readme_filename
            
            with open(readme_filepath, 'w', encoding='utf-8') as f:
                f.write(readme_content)
            
            print(f"✅ Generated README: {readme_filename}")
            
        except Exception as e:
            print(f"❌ Failed to generate metadata: {e}")
    
else:
    print("⚠️  Skipping metadata generation - insufficient data")
    metadata = None

## 7. Validate Export Format
Verify the exported files meet NotebookLM requirements and test sample imports.

In [ ]:
def validate_export_format(exported_files):
    """Validate exported files for NotebookLM compatibility"""
    
    validation_results = []
    
    if not exported_files:
        print("❌ No files to validate")
        return validation_results
    
    print("🔍 VALIDATION REPORT")
    print("=" * 50)
    
    for file_info in exported_files:
        filepath = file_info['filepath']
        
        # Check file existence
        if not filepath.exists():
            validation_results.append({
                'file': file_info['filename'],
                'status': 'FAILED',
                'issue': 'File not found',
                'recommendation': 'Re-run export process'
            })
            continue
        
        # Check file size (NotebookLM has limits)
        size_mb = file_info['size_kb'] / 1024
        size_status = 'PASS'
        size_issue = None
        
        if size_mb > 50:  # NotebookLM typical limit
            size_status = 'WARNING'
            size_issue = f'Large file ({size_mb:.1f}MB) may need splitting'
        elif size_mb < 0.1:
            size_status = 'WARNING'  
            size_issue = f'Very small file ({size_mb:.3f}MB) may lack content'
        
        # Check content format for text files
        if filepath.suffix == '.txt':
            try:
                with open(filepath, 'r', encoding='utf-8') as f:
                    content = f.read(1000)  # Read first 1KB
                
                # Check for proper text encoding
                content_status = 'PASS'
                content_issue = None
                
                if len(content.strip()) == 0:
                    content_status = 'FAILED'
                    content_issue = 'Empty content'
                elif not content.isprintable() and not any(c in content for c in ['\n', '\t', '\r']):
                    content_status = 'WARNING'
                    content_issue = 'May contain non-printable characters'
                
                # Check for structured format
                has_headers = bool(re.search(r'^#+ ', content, re.MULTILINE))
                structure_status = 'PASS' if has_headers else 'WARNING'
                structure_issue = None if has_headers else 'No markdown headers detected'
                
            except Exception as e:
                content_status = 'FAILED'
                content_issue = f'Content read error: {e}'
                structure_status = 'FAILED'
                structure_issue = 'Cannot analyze structure'
        
        elif filepath.suffix == '.json':
            try:
                with open(filepath, 'r', encoding='utf-8') as f:
                    json_data = json.load(f)
                
                content_status = 'PASS'
                content_issue = None
                structure_status = 'PASS'
                structure_issue = None
                
            except Exception as e:
                content_status = 'FAILED'
                content_issue = f'Invalid JSON: {e}'
                structure_status = 'FAILED'
                structure_issue = 'JSON parse error'
        
        else:
            content_status = 'UNKNOWN'
            content_issue = 'Unknown file format'
            structure_status = 'UNKNOWN'
            structure_issue = 'Cannot validate structure'
        
        # Overall status
        if content_status == 'FAILED' or structure_status == 'FAILED':
            overall_status = 'FAILED'
        elif content_status == 'WARNING' or structure_status == 'WARNING' or size_status == 'WARNING':
            overall_status = 'WARNING'
        else:
            overall_status = 'PASS'
        
        # Compile validation result
        result = {
            'file': file_info['filename'],
            'status': overall_status,
            'size_mb': size_mb,
            'size_status': size_status,
            'size_issue': size_issue,
            'content_status': content_status,
            'content_issue': content_issue,
            'structure_status': structure_status,
            'structure_issue': structure_issue,
            'notebooklm_ready': overall_status in ['PASS', 'WARNING']
        }
        
        validation_results.append(result)
        
        # Print result
        status_emoji = {'PASS': '✅', 'WARNING': '⚠️', 'FAILED': '❌', 'UNKNOWN': '❓'}
        print(f"{status_emoji[overall_status]} {file_info['filename']}")
        print(f"   Size: {size_mb:.1f}MB {status_emoji[size_status]}")
        print(f"   Content: {status_emoji[content_status]}")
        print(f"   Structure: {status_emoji[structure_status]}")
        
        if any([size_issue, content_issue, structure_issue]):
            print(f"   Issues: {', '.join(filter(None, [size_issue, content_issue, structure_issue]))}")
        
        print()
    
    # Summary
    total_files = len(validation_results)
    passed_files = len([r for r in validation_results if r['status'] == 'PASS'])
    warning_files = len([r for r in validation_results if r['status'] == 'WARNING'])
    failed_files = len([r for r in validation_results if r['status'] == 'FAILED'])
    ready_files = len([r for r in validation_results if r['notebooklm_ready']])
    
    print(f"📊 VALIDATION SUMMARY")
    print(f"   Total files: {total_files}")
    print(f"   ✅ Passed: {passed_files}")
    print(f"   ⚠️  Warnings: {warning_files}")
    print(f"   ❌ Failed: {failed_files}")
    print(f"   🎯 NotebookLM Ready: {ready_files}/{total_files}")
    
    # Recommendations
    print(f"\n💡 NOTEBOOKLM IMPORT RECOMMENDATIONS")
    
    if ready_files == total_files:
        print("🎉 All files are ready for NotebookLM import!")
        print("📋 Suggested import order:")
        print("   1. Start with the consolidated analysis for overview")
        print("   2. Import individual analysis documents for focused study")
        print("   3. Use the complete archive for detailed message context")
        print("   4. Reference metadata files for dataset understanding")
    else:
        print("⚠️  Some files need attention before NotebookLM import")
        
        for result in validation_results:
            if not result['notebooklm_ready']:
                print(f"   🔧 {result['file']}: {result['content_issue'] or result['structure_issue']}")
    
    return validation_results

# Validate exported files
if exported_files:
    validation_results = validate_export_format(exported_files)
    
    print(f"\n🎯 FINAL EXPORT STATUS")
    print(f"{'='*50}")
    
    ready_count = sum(1 for r in validation_results if r['notebooklm_ready'])
    total_count = len(validation_results)
    
    if ready_count == total_count and total_count > 0:
        print(f"✅ SUCCESS: All {total_count} files ready for NotebookLM analysis!")
        print(f"📂 Files location: {EXPORT_DIR}")
        print(f"\n🚀 Next steps:")
        print(f"   1. Navigate to {EXPORT_DIR}")
        print(f"   2. Upload files to NotebookLM")
        print(f"   3. Start with the consolidated analysis document")
        print(f"   4. Use the README file for guidance")
    else:
        print(f"⚠️  Status: {ready_count}/{total_count} files ready")
        print(f"📂 Files location: {EXPORT_DIR}")
        print(f"🔧 Review validation issues above")
        
else:
    print("❌ No files were exported to validate")

## 8. Execute Complete Pipeline

This section runs the entire export pipeline from data loading to validation.

In [ ]:
print("🚀 STARTING COMPLETE NOTEBOOKLM EXPORT PIPELINE")
print("=" * 60)

# Step 1: Load Discord data
print("\n📊 Step 1: Loading Discord data...")
try:
    messages_df, users_df, channels_df = load_discord_data()
    print(f"✅ Loaded {len(messages_df)} messages from {len(channels_df)} channels")
    print(f"✅ Found {len(users_df)} unique users")
except Exception as e:
    print(f"❌ Failed to load data: {e}")
    raise

# Step 2: Format data for NotebookLM
print("\n📝 Step 2: Formatting data for NotebookLM...")
try:
    formatted_data = format_for_notebooklm(messages_df, users_df, channels_df)
    print(f"✅ Generated {len(formatted_data)} formatted documents")
    
    # Show what we generated
    for doc_type, content in formatted_data.items():
        word_count = len(content.split())
        print(f"   📄 {doc_type}: {word_count:,} words")
        
except Exception as e:
    print(f"❌ Failed to format data: {e}")
    raise

# Step 3: Create structured documents
print("\n📚 Step 3: Creating structured documents...")
try:
    structured_docs = create_structured_documents(formatted_data, messages_df, users_df)
    print(f"✅ Created {len(structured_docs)} structured documents")
    
    for doc_name, doc_content in structured_docs.items():
        word_count = len(doc_content.split())
        print(f"   📑 {doc_name}: {word_count:,} words")
        
except Exception as e:
    print(f"❌ Failed to create structured documents: {e}")
    raise

# Step 4: Export to files
print("\n💾 Step 4: Exporting to files...")
try:
    exported_files = export_to_files(structured_docs, messages_df, users_df, channels_df)
    print(f"✅ Exported {len(exported_files)} files")
    
    total_size_mb = sum(f['size_kb'] for f in exported_files) / 1024
    print(f"📏 Total export size: {total_size_mb:.1f}MB")
    
    for file_info in exported_files:
        print(f"   📁 {file_info['filename']}: {file_info['size_kb']:.1f}KB")
        
except Exception as e:
    print(f"❌ Failed to export files: {e}")
    raise

# Step 5: Generate metadata
print("\n🏷️ Step 5: Generating metadata...")
try:
    metadata = generate_metadata(messages_df, users_df, channels_df)
    print(f"✅ Generated comprehensive metadata")
    
    # Show key statistics
    print(f"   📅 Date range: {metadata['data_summary']['date_range']}")
    print(f"   💬 Message count: {metadata['data_summary']['message_count']:,}")
    print(f"   👥 Active users: {metadata['data_summary']['active_users']}")
    
except Exception as e:
    print(f"❌ Failed to generate metadata: {e}")
    raise

# Step 6: Validate export format
print("\n🔍 Step 6: Validating export format...")
try:
    validation_results = validate_export_format(exported_files)
    
    ready_count = sum(1 for r in validation_results if r['notebooklm_ready'])
    total_count = len(validation_results)
    
    if ready_count == total_count:
        print(f"✅ All {total_count} files passed validation!")
    else:
        print(f"⚠️ {ready_count}/{total_count} files ready for NotebookLM")
        
except Exception as e:
    print(f"❌ Failed to validate export: {e}")
    raise

print(f"\n🎉 EXPORT PIPELINE COMPLETED SUCCESSFULLY!")
print(f"=" * 60)
print(f"📂 Export location: {EXPORT_DIR}")
print(f"📋 Files ready for NotebookLM analysis")
print(f"\n🎯 Next steps:")
print(f"   1. Navigate to the export directory")
print(f"   2. Review the README file for import guidance") 
print(f"   3. Upload documents to NotebookLM")
print(f"   4. Start analysis with the consolidated overview")